In [1]:
%reset -sf

In [2]:
from pprint import pprint
from collections import defaultdict, OrderedDict, namedtuple, deque
from random import choice, seed, randint, shuffle, random
from tqdm import tqdm
from itertools import repeat, product, combinations, cycle
from pathlib import Path
from string import ascii_lowercase
from gc import collect

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from deap import creator as ga_cr, base as ga_b, algorithms as ga_algo, tools as ga_t

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [4]:
path = Path('/kaggle') / 'input' / 'hashcode-2021-oqr-extension'
files = list(path.glob('*'))
files

[PosixPath('/kaggle/input/hashcode-2021-oqr-extension/hashcode.in'),
 PosixPath('/kaggle/input/hashcode-2021-oqr-extension/full_problem_description.pdf')]

In [5]:
def read_input_file(input_file_path):
    with open(input_file_path) as f:
        arr = f.readlines()
    return arr

inputs = read_input_file(files[0])

def parse_input(inputs):
    #with open(input_file_path) as f:
      #  arr = f.readlines()
    arr = inputs.copy()
    arr = arr[::-1]

    # first line
    line = arr.pop().split()
    duration, num_inters, num_streets, num_cars, fixed_score = [
        int(x) for x in line]
    # print(duration, num_inters, num_streets, num_cars, fixed_score)

    # parse streets with inters
    streets = {}
    for _ in range(num_streets):
        line = arr.pop().split()
        start = int(line[0])
        end = int(line[1])
        street_name = line[2]
        length = int(line[-1])
        streets[street_name] = {"start": start, "end": end, "length": length}
    # print(f"Unique Streets: {len(streets)}")

    # parse cars paths
    cars_paths = []
    for _ in range(num_cars):
        line = arr.pop().split()
        # [1:] because the first word is the length of the sequence
        sequence = line[1:]
        cars_paths.append(sequence)
    # print(f"Unique Car Paths: {len(cars_paths)}")

    return streets, cars_paths


class MutableValue:
    def __init__(self, val=None):
        self.val = val


Street = namedtuple(
    "Street",
    [
        "id",  # The index of the street
        "start",  # The inters object at the start of the street
        "end",  # The inters object at the end of the street
        "name",  # A str
        "duration",  # The length of the street in seconds
        "driving_cars",  # A dict mapping car ids (int) to remaining seconds
        "waiting_cars",  # A deque of car ids (int)
        "arrival_times",  # A dict mapping car ids (int) to their arrival times
        # A dict mapping car ids (int) to their departure times
        "departure_times"
        # You can compute the seconds that a car
        # was waiting at the end of the street
        # by subtracting the arrival time from the departure time.
    ],
)

Inters = namedtuple(
    "Inters",
    [
        "id",  # The index of the Inters
        "incomings",  # A deque of incoming Street objects
        "outgoings",  # A deque of outgoing Street objects
        # The Street object that currently has a green light.
        # Will be wrapped in a MutableValue with a "val" attribute to allow
        # mutating the value without mutating the namedtuple.
        "green_street",
        # An int representing the total number of waiting cars across all
        # incoming streets of this inters.
        # Will be wrapped in a MutableValue with a "val" attribute to allow
        # mutating the value without mutating the namedtuple.
        "num_waiting_cars",
        # The sum of green times of all incoming streets in the schedule.
        # Will be wrapped in a MutableValue with a "val" attribute to allow
        # mutating the value without mutating the namedtuple.
        "schedule_duration",
        # A list mapping (t mod schedule_duration.val) to the street object
        # that is green at time t.
        "green_street_per_t_mod",  # green_street_per_t_mod: name too long
        # A bool indicating whether the green_street ever needs to be
        # updated during the simulation (i.e., whether the schedule has
        # more than one street).
        # Will be wrapped in a MutableValue with a "val" attribute to allow
        # mutating the value without mutating the namedtuple.
        "needs_updates",
    ],
)

# We only use street indices and inters indices here to allow
# fast deep-copies of a schedule for testing out and reverting modifications.
Schedule = namedtuple(
    "Schedule",
    [
        "i_inters",  # The index of the inters
        "order",  # A list of street ids
        "green_times",  # A dict mapping street ids to green times (seconds)
    ],
)


def read_input_file_(input_file_path):
    with open(input_file_path) as f:
        lines = deque(f.readlines())
    return lines

inputs_ = read_input_file_(files[0])


def parse_input2(inputs_):
    #with open(input_file_path) as f:
     #   lines = deque(f.readlines())

    # Parse the first line
    lines = inputs_.copy()
    total_duration, num_interss, num_streets, num_cars, bonus_points = map(
        int, lines.popleft().split()
    )

    # Create empty interss
    interss = tuple(
        Inters(
            id=i,
            incomings=deque(),
            outgoings=deque(),
            green_street=MutableValue(),
            num_waiting_cars=MutableValue(0),
            green_street_per_t_mod=[],
            schedule_duration=MutableValue(),
            needs_updates=MutableValue(False),
        )
        for i in range(num_interss)
    )

    # Parse the streets
    streets = []
    name_to_street = {}
    for i_street in range(num_streets):
        line = lines.popleft().split()
        start, end = map(int, line[:2])
        name = line[2]
        duration = int(line[3])
        street = Street(
            id=i_street,
            start=interss[start],
            end=interss[end],
            name=name,
            duration=duration,
            driving_cars={},
            waiting_cars=deque(),
            arrival_times={},
            departure_times={},
        )
        name_to_street[name] = street
        interss[start].outgoings.append(street)
        interss[end].incomings.append(street)
        streets.append(street)

    # Parse the paths
    paths = []
    for i_car in range(num_cars):
        line = lines.popleft().split()
        path_length = int(line[0])
        path = line[1:]
        assert len(path) == path_length
        path = deque(name_to_street[name] for name in path)
        paths.append(path)

    return (total_duration, bonus_points, interss,
            streets, name_to_street, paths)

In [6]:
def create_inters(streets, cars_paths, cars_paths_order='descending'):
    if cars_paths_order == 'descending':
        cars_paths = reversed(sorted(cars_paths, key=len))
    if cars_paths_order == 'ascending':
        cars_paths = sorted(cars_paths, key=len)
    if cars_paths_order == 'random':
        shuffle(cars_paths)

    inters_streets = defaultdict(set)
    for car_path in cars_paths:
        for street in car_path:
            inters_streets[streets[street]["end"]].add(street)
    # print(f"Unique Intersections: {len(inters_streets)}")

    return inters_streets


# NOTE: There are some intersections never reached by cars paths
# So, they should not be accounted for in schedules, otherwise error!


def create_time_fractions(interss, light_times):
    # each inters gets a light
    # each light gets own schedule
    # very granular!
    #inters_sched = defaultdict(dict)
    #for i, inters in enumerate(interss):
     #    for j, street in enumerate(interss[inters]):
      #          inters_sched[inters][street] = randint(1, 3)
                
    inters_sched = defaultdict(dict)
    for inters, time_fraction in zip(interss, light_times):
        for street in interss[inters]:
            inters_sched[inters][street] = time_fraction

    return inters_sched


def set_schedules(inters_streets, time_fractions):
    # Set schedules
    schedules = []
    for inters, streets in inters_streets.items():
        cycle = []
        for street in streets:
            # time_fraction = 1  # int(randint(1, 5))
            # time_fraction = 1
            cycle.append([inters, street, time_fractions[inters][street]])
        schedules.append(cycle)
    return schedules


def format_schedules(schedules):
    # Parse solution into submission
    res = []
    res.append([len(schedules)])  # requirement
    for cycle in schedules:
        res.append([cycle[0][0]])  # requirement: inters ID
        res.append([len(cycle)])  # requirement: lenght of inters
        for inters, street, time_fraction in cycle:
            res.append([street, time_fraction])
    res1 = "\n".join(" ".join([str(x) for x in row]) for row in res)
    # with open("submission.csv", "w") as text_file:
    #     text_file.write(res1)
    res2 = [" ".join([str(x) for x in row]) for row in res]
    return res1, res2


def parse_schedules(output_file_path, name_to_street):
    # with open(output_file_path) as f:
    #     lines = f.readlines()
    #     lines = deque(lines)
    # adusted to consume from variables,
    # instead of file path
    lines = deque(output_file_path)
    num_schedules = int(lines.popleft())
    schedules = []
    for i_schedule in range(num_schedules):
        i_inters = int(lines.popleft())
        num_incomings = int(lines.popleft())
        order = []
        green_times = {}
        for i_incoming in range(num_incomings):
            street_name, green_time = lines.popleft().split()
            green_time = int(green_time)
            street = name_to_street[street_name]
            order.append(street.id)
            green_times[street.id] = green_time

        schedule = Schedule(i_inters=i_inters, order=order,
                            green_times=green_times)
        schedules.append(schedule)
    return schedules


def reset(streets, interss):
    # Reinitialize mutable data structures
    for street in streets:
        street.driving_cars.clear()
        street.waiting_cars.clear()
        street.arrival_times.clear()
        street.departure_times.clear()

    for inters in interss:
        inters.green_street.val = None
        inters.num_waiting_cars.val = 0
        inters.green_street_per_t_mod.clear()
        inters.schedule_duration.val = None
        inters.needs_updates.val = False


def grade(schedules, streets, interss, paths,
          total_duration, bonus_points):
    reset(streets, interss)

    # We will consume the deques in the paths list. Save a copy of them
    # for later to reset the paths after the simulation.
    paths_copy = [path.copy() for path in paths]

    # Iterate through the schedules and initialize the interss.
    inters_ids_with_schedules = set()
    for schedule in schedules:
        inters = interss[schedule.i_inters]
        inters_ids_with_schedules.add(inters.id)
        first_street = streets[schedule.order[0]]
        inters.green_street.val = first_street
        inters.needs_updates.val = len(schedule.order) > 1
        schedule_duration = 0
        green_street_per_t_mod = inters.green_street_per_t_mod
        for street_id in schedule.order:
            green_time = schedule.green_times[street_id]
            for _ in range(green_time):
                green_street_per_t_mod.append(streets[street_id])
            schedule_duration += green_time
        inters.schedule_duration.val = schedule_duration

    # inters_ids_with_waiting_cars is restricted to interss
    # with schedules
    inters_ids_with_waiting_cars = set()
    for i_car, path in enumerate(paths):
        street = path.popleft()
        street.waiting_cars.append(i_car)
        if street.end.id in inters_ids_with_schedules:
            inters_ids_with_waiting_cars.add(street.end.id)
        street.end.num_waiting_cars.val += 1

    street_ids_with_driving_cars = set()
    score = 0

    # Main simulation loop
    for t in range(total_duration):

        # Drive across interss
        # Store the ids of interss that
        # don't have waiting cars after this.
        inters_ids_to_remove = set()
        for i_inters in inters_ids_with_waiting_cars:
            inters = interss[i_inters]

            if inters.needs_updates.val:
                # Update the green street
                t_mod = t % inters.schedule_duration.val
                inters.green_street.val = inters.green_street_per_t_mod[t_mod]

            green_street = inters.green_street.val
            waiting_cars = green_street.waiting_cars
            if len(waiting_cars) > 0:
                # Drive across the inters
                waiting_car = waiting_cars.popleft()
                green_street.departure_times[waiting_car] = t
                next_street = paths[waiting_car].popleft()
                next_street.driving_cars[waiting_car] = next_street.duration
                street_ids_with_driving_cars.add(next_street.id)

                inters.num_waiting_cars.val -= 1
                if inters.num_waiting_cars.val == 0:
                    inters_ids_to_remove.add(i_inters)

        inters_ids_with_waiting_cars.difference_update(inters_ids_to_remove)

        # Drive across roads
        # Store the ids of streets that don't have driving cars after this.
        street_ids_to_remove = set()
        for i_street in street_ids_with_driving_cars:
            street = streets[i_street]
            driving_cars = street.driving_cars
            for car in list(driving_cars):
                # Update the "time to live" of this car,
                # i.e. the remaining driving seconds.
                ttl = driving_cars[car]
                ttl -= 1
                if ttl < 0:
                    raise ValueError
                elif ttl == 0:
                    # Reached the end of the street
                    del driving_cars[car]
                    if len(paths[car]) == 0:
                        # FINISH
                        score += bonus_points
                        score += total_duration - t - 1

                    else:
                        street.waiting_cars.append(car)
                        street.end.num_waiting_cars.val += 1
                        street.arrival_times[car] = t + 1
                        inters_id = street.end.id
                        if inters_id in inters_ids_with_schedules:
                            inters_ids_with_waiting_cars.add(inters_id)
                else:
                    # The car is still driving on the street
                    driving_cars[car] = ttl
            if len(driving_cars) == 0:
                street_ids_to_remove.add(i_street)
        street_ids_with_driving_cars.difference_update(street_ids_to_remove)

    # We are done with the simulation. Restore the paths.
    for i_path in range(len(paths)):
        paths[i_path] = paths_copy[i_path]

    return score

def write_best_score(score, schedules):
    with open(f"submission_score_{score:,}.csv", "w") as text_file:
        text_file.write(schedules)

In [7]:
class GA_Hash_Traffic_Light():
    def __init__(self, 
                 #params, 
                 eval_func,
                 eval_weights,
                 #
                 #input_file,
                 inputs,
                 inputs_,
                 #
                 lb_light_time=1,
                 ub_light_time=10,
                 #
                 #inters_streets=inters_streets,
                 #name_to_i_street=name_to_i_street, 
                 #streets2=streets2,
                 #inters=inters,
                 #paths=paths,
                 #duration=duration,
                 #bonus_points=bonus_points,
                 #
                 sel_tournsize=2, 
                 cx_uniform_prob=0.5, 
                 mut_shuffle_idx_prob=0.1, 
                 n_pop=75, 
                 n_gen=75, 
                 n_hof=1, 
                 cx_prob=0.5, 
                 mut_prob=0.1, 
                 n_jobs=4
                ):
        #self.params = params
        self.eval_func = eval_func
        self.eval_weights = eval_weights
        
        #self.input_file = input_file
        self.inputs = inputs
        self.inputs_ = inputs_
        self.lb_light_time = lb_light_time
        self.ub_light_time = ub_light_time
        
        #self.inters_streets = inters_streets
        #self.name_to_i_street = name_to_i_street
        #self.streets2 = streets2
        #self.inters = inters
        #self.paths = paths
        #self.duration = duration
        #self.bonus_points = bonus_points
        
        self.sel_tournsize = sel_tournsize
        self.cx_uniform_prob = cx_uniform_prob
        self.mut_shuffle_idx_prob = mut_shuffle_idx_prob
        self.n_pop = n_pop
        self.n_gen = n_gen
        self.n_hof = n_hof
        self.cx_prob = cx_prob
        self.mut_prob = mut_prob
        
        self.n_jobs = n_jobs

        self._create_fitness_and_indiv()
        self._register_indiv_and_pop_generators()
        self._register_eval_func()
        self._register_selection_crossover_mutation_methods()

    def _create_fitness_and_indiv(self):
        """Create GA individual and fitness entities (classes)"""
        ga_cr.create('Fitness', ga_b.Fitness, weights=self.eval_weights)
        ga_cr.create('Individual', list, fitness=ga_cr.Fitness)
        print('GA entities created')

    def _gen_params_to_ga(self):
        """Generate index for each param for individual"""
        idxs = [randint(self.lb_light_time, self.ub_light_time) for _ in range(7999)]
        return idxs
    
    def _register_indiv_and_pop_generators(self):
        """Register GA individual and population generators"""
        self.tb = ga_b.Toolbox()

        if self.n_jobs > 1:
            from multiprocessing import Pool
            pool = Pool()
            self.tb.register("map", pool.map)

        self.tb.register("individual", ga_t.initIterate, ga_cr.Individual, self._gen_params_to_ga)
        #print('indiv', self.tb.individual())
        self.tb.register("population", ga_t.initRepeat, list, self.tb.individual)
        #print('population', self.tb.population(n=2))
        print('GA entities\' methods registered')
        
    def _register_eval_func(self):
        """Set GA evaluate individual function"""
        self.tb.register("evaluate",
                        self.eval_func,
                        inputs=self.inputs,
                        inputs_=self.inputs_,
                        )
        #print(list(self.tb.evaluate(indiv) for indiv in self.tb.population(3)))
        print('GA eval function registered')
    
    def _register_selection_crossover_mutation_methods(self):
        self.tb.register("select", ga_t.selTournament, tournsize=self.sel_tournsize)
        self.tb.register("mate", ga_t.cxUniformPartialyMatched, indpb=self.cx_uniform_prob)  # , (cxUniform, indpb=self.cx_uniform_prob) (PartialyMatched) (cxUniformPartialyMatched, indpb=self.cx_uniform_prob)
        self.tb.register("mutate", ga_t.mutUniformInt, low=self.lb_light_time, up=self.ub_light_time, indpb=self.mut_shuffle_idx_prob)
        print('GA sel-cx-mut methods registered')
        
    def run_ga_search(self):
        """GA Search"""
        pop = self.tb.population(n=self.n_pop)
        hof = ga_t.HallOfFame(self.n_hof)

        # Stats stdout
        stats = ga_t.Statistics(lambda ind: ind.fitness.values )
        #stats1 = ga_t.Statistics(lambda ind: ind.fitness.values[0] )
        #stats2 = ga_t.Statistics(lambda ind: ind.fitness.values[1] )
        #stats3 = ga_t.Statistics(lambda ind: ind.fitness.values[2] )
        stats = ga_t.MultiStatistics(traffic_score=stats)
        stats.register("avg", np.mean)
        #stats.register("std", np.std)
        #stats.register("min", np.min)
        stats.register("max", np.max)

        # History
        #hist = tools.History()
        #toolbox.decorate("select", hist.decorator)
        #tb.decorate("mate", hist.decorator)
        #tb.decorate("mutate", hist.decorator)
        #hist.update(pop)

        # GA Run
        pop, log = ga_algo.eaSimple(pop, self.tb, cxpb=self.cx_prob, 
                                    mutpb=self.mut_prob, ngen=self.n_gen, 
                                    stats=stats, halloffame=hof, verbose=True)
        
        # Convert back params
        #hof_ = {}
        #for i in range(self.n_hof):
         #   hof_['hof_' + str(i)] = self._ga_to_params(hof[i])

        return pop, log, hof
    
    def _ga_to_params(self, idx_params):
        """Convert back idx to params"""
        res = {}
        for (k,v), idx in zip(self.padded_params.items(), idx_params):
            res[k] = v[idx]
        return res

In [8]:
def hash_eval_indiv(individual, inputs, inputs_):
    """Evaluate individual's genes (estimator's params)"""
    # Containers
    streets, cars_paths = parse_input(inputs)
    duration, bonus_points, inters, streets2, name_to_i_street, paths = parse_input2(inputs_)

    # Create intersections
    inters_streets = create_inters(streets, cars_paths)

    # Create schedules times
    time_fractions = create_time_fractions(inters_streets, individual)

    # Schedules
    schedules = set_schedules(inters_streets, time_fractions)
    schedules_str, schedules_file = format_schedules(schedules)

    # Evaluation
    schedules_for_grade = parse_schedules(schedules_file, name_to_i_street)
    score = grade(schedules_for_grade, streets2, inters, paths, duration, bonus_points)
    
    collect()

    return (score,)

hash_weights = (1,)

In [9]:
ga_hash = GA_Hash_Traffic_Light(hash_eval_indiv, 
                                hash_weights,
                                #
                                inputs,
                                inputs_,
                                #
                                ub_light_time=3
                               )
pop, log, hof = ga_hash.run_ga_search()

GA entities created
GA entities' methods registered
GA eval function registered
GA sel-cx-mut methods registered
   	      	                traffic_score                 
   	      	----------------------------------------------
gen	nevals	avg        	gen	max        	nevals
0  	75    	3.09994e+06	0  	3.26292e+06	75    
1  	42    	3.14701e+06	1  	3.26292e+06	42    
2  	48    	3.18567e+06	2  	3.29029e+06	48    
3  	42    	3.21975e+06	3  	3.37752e+06	42    
4  	36    	3.25208e+06	4  	3.41014e+06	36    
5  	44    	3.27544e+06	5  	3.42006e+06	44    
6  	42    	3.2946e+06 	6  	3.42006e+06	42    
7  	36    	3.31231e+06	7  	3.42006e+06	36    
8  	34    	3.33372e+06	8  	3.43249e+06	34    
9  	46    	3.34453e+06	9  	3.42017e+06	46    
10 	37    	3.36746e+06	10 	3.47743e+06	37    
11 	45    	3.37844e+06	11 	3.47743e+06	45    
12 	42    	3.39162e+06	12 	3.47743e+06	42    
13 	34    	3.41247e+06	13 	3.47743e+06	34    
14 	39    	3.43065e+06	14 	3.49748e+06	39    
15 	40    	3.4445e+06 	15 	3.50149e

In [10]:
np.array(hof[0])

array([1, 1, 2, ..., 2, 3, 3])

In [11]:
# Hash with best HoF

# Containers
streets, cars_paths = parse_input(inputs)
duration, bonus_points, inters, streets2, name_to_i_street, paths = parse_input2(inputs_)
    
# Create intersections
inters_streets = create_inters(streets, cars_paths)

# Create schedules times
time_fractions = create_time_fractions(inters_streets, light_times=hof[0])
    
# Schedules
schedules = set_schedules(inters_streets, time_fractions)
schedules_str, schedules_file = format_schedules(schedules)

# Evaluation
schedules_for_grade = parse_schedules(schedules_file, name_to_i_street)
score = grade(schedules_for_grade, streets2, inters, paths, duration, bonus_points)
score

3684760

In [12]:
# Write
write_best_score(score, schedules_str)

In [13]:
if False:
    # one light schedule for all inters!
        # OPT_TIME_FRACTION = int(trial.suggest_float(
        #     'opt_time_fraction', lower_time_fraction, upper_time_fraction))
        # inters_sched = defaultdict(dict)
        # for inters in interss:
        #     for j, street in enumerate(interss[inters]):
        #         inters_sched[inters][street] = OPT_TIME_FRACTION

        if grouping:
            time_fractions = []
            for i in range(grouping):
                OPT_TIME_FRACTION = int(trial.suggest_float(
                    'OPT_TIME_FRACTION_' + str(i),
                    lower_time_fraction,
                    upper_time_fraction))
                time_fractions.append(OPT_TIME_FRACTION)
            inters_sched = defaultdict(dict)
            for inters, time_fraction in zip(interss, cycle(time_fractions)):
                for j, street in enumerate(interss[inters]):
                    inters_sched[inters][street] = time_fraction

        ###################
        # num_inc_strs = []
        # for inters in interss:
        #     num_inc_strs.append(len(list(interss[inters])))
        # print(f'mean street per inters:', {array(num_inc_strs).mean()})
        ###################

        # each inters gets light
        # each light has same schedule
        # time_fractions = []
        # for i in range(len(interss)):
        #     name = "OPT_TIME_FRACTION_" + str(i)  # + '_' + str(j)
        #     # OPT_TIME_FRACTION = trial.suggest_int(
        #     #     name, lower_time_fraction, upper_time_fraction)
        #     OPT_TIME_FRACTION = int(trial.suggest_float(
        #         name, lower_time_fraction, upper_time_fraction))
        #     time_fractions.append(OPT_TIME_FRACTION)
        # inters_sched = defaultdict(dict)
        # for inters, time_fraction in zip(interss, cycle(time_fractions)):
        #     for j, street in enumerate(interss[inters]):
        #         inters_sched[inters][street] = time_fraction

        # in-between previous and next one

In [14]:

"""from torch import optim
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from torch import (load, amax as pt_amax, max as pt_max, ones, save, no_grad, stack, numel, tensor, 
                   manual_seed, sigmoid, tanh, add, mul, sub, div, amin as pt_amin, cat,
                  maximum, minimum, device, cuda, rand, prod, median, log as pt_log, round as pt_round,
                  isnan, flatten, mean)
from torch import nn
from torch.nn import functional as F
from torch.optim import Adam
#from torchviz import make_dot
import torchvision
import torchvision.transforms as transforms"""

'from torch import optim\nfrom torch.utils.data import DataLoader, Dataset\nfrom torchvision.datasets import ImageFolder\nfrom torch import (load, amax as pt_amax, max as pt_max, ones, save, no_grad, stack, numel, tensor, \n                   manual_seed, sigmoid, tanh, add, mul, sub, div, amin as pt_amin, cat,\n                  maximum, minimum, device, cuda, rand, prod, median, log as pt_log, round as pt_round,\n                  isnan, flatten, mean)\nfrom torch import nn\nfrom torch.nn import functional as F\nfrom torch.optim import Adam\n#from torchviz import make_dot\nimport torchvision\nimport torchvision.transforms as transforms'

In [15]:
def _pad_params(self):
        """Pad params for crossover shuffle idx method"""
        assert isinstance(self.params, dict), 'Params must be a dict, i.e. estimator.get_params()'
        params_count = {k: len(v) for k,v in self.params.items()}
        max_length, max_key = -99, ''
        for k, v in params_count.items():
            if v <= max_length:
                continue
            else:
                max_key = k
                max_length = v
        assert isinstance(max_length, int), 'The max length between all params must be an int'
        # cycle through params for max length param, otherwise infinite cycle
        values_padded = (cycle(v) if k!=max_key else v for k,v in self.params.items())
        values_padded = zip(*values_padded)  # ('a', 1, 14), ('b', 2, 16), ('c', 3, 16) ...
        values_padded = zip(*values_padded)  # ('a', 'b', 'c'), (1, 2, 3), (14, 15, 16)...
        padded_params = {}
        for k, v in zip(self.params, values_padded):
            padded_params[k] = v
        self.padded_params = padded_params
        print('Params padded')